OpenAI's gym is now deprecated, https://gymnasium.farama.org should be used instead.

## Installation

To run this you need several packages. First of all, you need anaconda, which you most likely already have if you're viewing this through jupyter. If not then check the readme on the class page.

System requirements: This should work on all operating systems (Linux, Mac, and Windows). However, several of the environments in the OpenAI-gym require additional simulators which don't aren't easy to get on Windows. In any case, it is strongly recommended that you use Linux, although you should be ok with Mac. (HINT: if you're on Windows check out the Windows Subsystem for Linux (WSL), although it'll make visualizing your policies a little tricky).

Then install the following packages (using conda or pip):

- pytorch --> `conda install pytorch -c pytorch`
- gym --> `pip install gym`
- gym (the cool environments, doesnt work on Windows) --> `pip install gym[all]`
(When install gym[all] don't worry if the mujoco installation doesn't work. That's a more advanced 3D physics simulator that has to be set up separately (see website). Anyway, we don't need it necessarily).

In [ ]:
# If you're using colab, this will install the necessary packages!
!pip install torch
!pip install gymnasium
!pip install numpy==1.24.1
!pip install "gymnasium[classic-control]"
!pip install swig
!pip install "gymnasium[box2d]"
!pip install "gymnasium[atari, accept-rom-license]"
!pip install shimmy


In [ ]:
import sys, os, time
import numpy as np
import matplotlib.pyplot as plt
import torch
torch.manual_seed(573)
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.multiprocessing as mp
from torch import distributions
from torch.distributions import Categorical
from itertools import islice
import ale_py
import shimmy

import gymnasium as gym

# Introduction
Welcome to the RL playground. Your task is to implement the REINFORCE and A3C algorithm to solve various OpenAI-gym environments. If you are not familiar with OpenAI-gym, stop reading and visit https://gym.openai.com/envs/ to see all the tasks you can try to solve.

In this homework, we will only look at tasks with a discrete (and small) action space. That being said, both algorithms can be modified slightly to work on tasks with continuous action spaces. For full credit you must fill in the code below so you achieve an average total reward per episode on the cartpole task (CartPole-v1) of at least 499 (for an episode length of 500) for both REINFORCE and A3C. Then you must apply your code to any one other environment in OpenAI-gym, and plot and compare the learning curves (average total reward per episode vs number of episodes trained on) between REINFORCE and A3C (where at least one of the algorithms shows significant improvement from initialization).

Below there's an overview of what every iteration will look like, regardless of whether you want to train or evaluate your agent.

In [ ]:
from rlhw_util import * # <-- look whats inside here - it could save you a lot of work!

def run_iteration(mode, N, agent, gen, horizon=None, render=False):
    train = mode == 'train'
    if train:
        agent.train()
    else:
        agent.eval()
    states, actions, rewards, prev_states = zip(*[gen(horizon=horizon, render=render) for _ in range(N)])
    loss = None
    if train:
        loss = agent.learn(states, actions, rewards, prev_states)

    reward = sum([r.sum() for r in rewards]) / N

    return reward, loss

## The Actor

We need to learn a policy which, given some state, outputs a distribution over all possible actions. As this is deep RL, we'll use a deep neural network to turn the observed state into the requisite action distribution. From this action distribution we can choose what action to take using `get_action`. Pytorch, brilliant as it is, makes our task incredibly easy, as we can use the `torch.distributions.Categorical` class for sampling.

You can experiment with all sorts of network architectures, but remember this is RL, not image classification on ImageNet, so you probably won't need a very deep network (HINT: look below at the state and action dimensionality to get a feel for the task).

In [ ]:
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Actor, self).__init__()
        
        self.fc1 = nn.Linear(state_dim, 64)
        self.fc2 = nn.Linear(64, action_dim)
        
    def forward(self, state):
        
        x = torch.relu(self.fc1(state))
        return torch.softmax(self.fc2(x), dim=-1)

    def get_policy(self, state):
        return Categorical(self(state))

    def get_action(self, state, greedy=None):
        if greedy is None:
            greedy = not self.training

        policy = self.get_policy(state)
        return MLE(policy) if greedy else policy.sample()

## The REINFORCE Agent

The Actor defines our policy, but we also have to define how and when we'll be updating our policy, which brings us to the agent. The agent will house the policy (an `Actor`), and can then be used to generate rollouts (using `forward()`) or update the policy given a list of rollouts (using `learn()`).

The REINFORCE algorithm naively uses the returns directly to weight the gradients, however this makes the variance in the policy gradient estimation very large. As a result, we will use a baseline which is a linear model which takes in a state and outputs the return (sounds like a value function, right?). Except we're not going to train our baseline using gradient descent, instead we'll just solve the linear system analytically in every iteration, and use the solution in the next iteration. Don't worry about training/updating the baseline, but you do have to use it in the right way. (Optional experiment: try removing the baseline and see how performance changes)

In [ ]:
class REINFORCE(nn.Module):
    
    def __init__(self, state_dim, action_dim, discount=0.97, lr=1e-3, weight_decay=1e-4):
        super(REINFORCE, self).__init__()
        self.actor = Actor(state_dim, action_dim)
        
        self.baseline = nn.Linear(state_dim, 1)
        
        self.optimizer = optim.RMSprop(self.actor.parameters(), lr=lr, weight_decay=weight_decay)
        
        self.discount = discount
        
    def forward(self, state):
        return self.actor.get_action(state)
    
    def learn(self, states, actions, rewards, _):
        '''
        Takes in three arguments each of which is a list with equal length. Each element in the list is a 
        pytorch tensor with 1 row for every step in the episode, and the columns are state_dim, action_dim, 
        and 1, respectively.
        '''
        
        returns = [compute_returns(rs, discount=self.discount) for rs in rewards]
        
        states, actions, returns = torch.cat(states), torch.cat(actions), torch.cat(returns)
        
        # Compute advantages (returns - baseline predictions)
        with torch.no_grad():
            baseline_values = self.baseline(states).squeeze()
            advantages = returns - baseline_values

        # Compute the log probabilities of the actions taken
        log_probs = []
        for state, action in zip(states, actions):
            policy = self.actor.get_policy(state)
            log_probs.append(policy.log_prob(action).unsqueeze(0))
        log_probs = torch.cat(log_probs, dim=0)

        # Compute the policy loss (negative of the weighted log-probabilities)
        policy_loss = -(log_probs * advantages).mean()

        # Update the actor's parameters
        self.optimizer.zero_grad()
        policy_loss.backward()
        self.optimizer.step()
        
        error = F.mse_loss(self.baseline(states).squeeze(), returns).detach()
        solve(states, returns, out=self.baseline)
        #error = F.mse_loss(self.baseline(states).squeeze(), returns).detach()
        
        return error.item() # Returns a rough estimate of the error in the baseline (dont worry about this too much)

## The Critic

Now we can introduce a critic, which is essentially a value function to estimate the expected discounted reward of a state.

In [ ]:
class Critic(nn.Module):
    def __init__(self, state_dim):
        super(Critic, self).__init__()
        
        self.fc1 = nn.Linear(state_dim, 64)  # First fully connected layer
        self.fc2 = nn.Linear(64, 1)  # Output layer to estimate value

    def forward(self, state):
        
        x = torch.relu(self.fc1(state))  # Apply ReLU to the first layer
        value = self.fc2(x)  # Get the state value
        return value

## The A3C Agent

Now we can put the actor and critic together using the A3C algorithm. It turns out, the tasks in the gym are all so simple that there is essentially no gain in parallelization, so technically we're implementing A2C (no async), but the RL part is the same.

In [ ]:
class A3C(nn.Module):
    
    def __init__(self, state_dim, action_dim, discount=0.97, lr=1e-3, weight_decay=1e-4):
        super(A3C, self).__init__()
        self.actor = Actor(state_dim, action_dim)
        self.critic = Critic(state_dim)
        
        self.actor_optimizer = torch.optim.RMSprop(self.actor.parameters(), lr=lr, weight_decay=weight_decay)
        self.critic_optimizer = torch.optim.RMSprop(self.critic.parameters(), lr=lr, weight_decay=weight_decay)
        
        self.discount = discount
        
    def forward(self, state):
        return self.actor.get_action(state)
    
    def learn(self, states, actions, rewards, _):
        
        returns = [compute_returns(rs, discount=self.discount) for rs in rewards]
        
        states, actions, returns = torch.cat(states), torch.cat(actions), torch.cat(returns)
        
        values = self.critic(states).squeeze()
        advantages = returns - values.detach() 

        # Actor loss (policy gradient with advantages)
        log_probs = []
        for state, action in zip(states, actions):
            policy = self.actor.get_policy(state)
            log_probs.append(policy.log_prob(action).unsqueeze(0))
        log_probs = torch.cat(log_probs, dim=0)
        actor_loss = -(log_probs * advantages).mean()

        # Critic loss (value function regression)
        critic_loss = F.mse_loss(values, returns)

        # Update actor network
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Update critic network
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        return actor_loss.item(), critic_loss.item()

## PPO

In [ ]:
class PPO(nn.Module):
    def __init__(self, state_dim, action_dim, discount=0.97, lr=1e-3, weight_decay=1e-4, clip_epsilon=0.2, 
                 actor_epochs=10, critic_epochs=10, max_grad_norm=0.5):
        super(PPO, self).__init__()

        self.actor = Actor(state_dim, action_dim)
        self.critic = Critic(state_dim)

        self.actor_optimizer = torch.optim.RMSprop(self.actor.parameters(), lr=lr, weight_decay=weight_decay)
        self.critic_optimizer = torch.optim.RMSprop(self.critic.parameters(), lr=lr, weight_decay=weight_decay)

        self.discount = discount
        self.clip_epsilon = clip_epsilon
        self.actor_epochs = actor_epochs
        self.critic_epochs = critic_epochs
        self.max_grad_norm = max_grad_norm

    def forward(self, state):
        return self.actor.get_action(state)

    def get_action(self, state, greedy=None):
        if greedy is None:
            greedy = not self.training

        policy = self.actor.get_policy(state)
        return MLE(policy) if greedy else policy.sample()

    def learn(self, states, actions, rewards, _):
        returns = [compute_returns(rs, discount=self.discount) for rs in rewards]

        states, actions, returns = torch.cat(states), torch.cat(actions), torch.cat(returns)
        advantages = returns - self.critic(states).squeeze().detach()

        old_policies = []
        for state, action in zip(states, actions):
            policy = self.actor.get_policy(state)
            old_policies.append(policy.log_prob(action).unsqueeze(0))
        old_policies = torch.cat(old_policies, dim=0).detach()

        for _ in range(self.actor_epochs):
            log_probs = []
            for state, action in zip(states, actions):
                policy = self.actor.get_policy(state)
                log_probs.append(policy.log_prob(action).unsqueeze(0))
            log_probs = torch.cat(log_probs, dim=0)

            ratios = torch.exp(log_probs - old_policies)
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1 - self.clip_epsilon, 1 + self.clip_epsilon) * advantages

            actor_loss = -torch.min(surr1, surr2).mean()

            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            nn.utils.clip_grad_norm_(self.actor.parameters(), self.max_grad_norm)
            self.actor_optimizer.step()

        for _ in range(self.critic_epochs):
            values = self.critic(states).squeeze()
            critic_loss = F.mse_loss(values, returns)

            self.critic_optimizer.zero_grad()
            critic_loss.backward()
            nn.utils.clip_grad_norm_(self.critic.parameters(), self.max_grad_norm)
            self.critic_optimizer.step()

        return actor_loss.item(), critic_loss.item()

## DQN

In [ ]:
from collections import deque
from torch import tensor
import random

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def add(self, state, action, reward, next_state):
        self.buffer.append((state, action, reward, next_state))
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states = zip(*batch)
        return torch.cat(states), torch.cat(actions), torch.cat(rewards), torch.cat(next_states)
    
    def size(self):
        return len(self.buffer)

class DQN(nn.Module):
    def __init__(self, state_dim, action_dim, lr=1e-3, weight_decay=1e-4, gamma=0.99, epsilon=0.9, epsilon_decay=0.999, min_epsilon=0.05, buffer_capacity=10000, batch_size=128, tau = 0.005):
        super(DQN, self).__init__()
        self.q_network = nn.Sequential(
            nn.Linear(state_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim)
        )

        self.target_q_network = nn.Sequential(
            nn.Linear(state_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim)
        )
        
        self.optimizer = optim.AdamW(self.q_network.parameters(), amsgrad=True)

        # Set hyperparameters
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.min_epsilon = min_epsilon
        self.batch_size = batch_size
        self.tau = tau

        # Replay buffer
        self.replay_buffer = ReplayBuffer(buffer_capacity)

        # Copy initial weights to the target network
        self.target_q_network.load_state_dict(self.q_network.state_dict())

    def forward(self, state):
        """Given a state, predict Q-values for all actions."""
        self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)
        rndsmp = torch.rand(1).item()
        if rndsmp > self.epsilon:
            with torch.no_grad():
                q_values = self.q_network(state)
                action = tensor([q_values.argmax()])
        else:
            return torch.randint(0, self.q_network[-1].out_features, (1,))
        
        return action

    def get_action(self, state, greedy=None):
        """Select an action using epsilon-greedy policy."""
        self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)
        rndsmp = torch.rand(1).item()
        if rndsmp > self.epsilon:
            with torch.no_grad():
                q_values = self.q_network(state)
                action = q_values.argmax().item()
        else:
            q_values = self.q_network(state)
            action = q_values.argmax().item()
            action = torch.randint(0, self.q_network[-1].out_features, (1,)).item()
        
        return action

    def learn(self, states, actions, rewards, prev_states): # we will just sample states, actions, and rewards, but we need to appease the syntax
        """Update the Q-network using the DQN loss."""
        states, actions, rewards, prev_states = torch.cat(states), torch.cat(actions), torch.cat(rewards), torch.cat(prev_states)
        self.replay_buffer.add(prev_states, actions, rewards, states)
        if self.replay_buffer.size() < self.batch_size:
            return None
        # Sample from replay buffer
        states, actions, rewards, next_states = self.replay_buffer.sample(self.batch_size)

        
        # Compute target Q-values for next states
        with torch.no_grad():
            max_next_q_values = self.target_q_network(next_states).max(dim=1).values
            targets = rewards + self.gamma * max_next_q_values

        # Compute Q-values for current states
        q_values = self.q_network(states).gather(1, actions)

        # Compute the loss
        loss = F.mse_loss(q_values, targets.unsqueeze(1))

        # Update the Q-network
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        target_net_state_dict = self.target_q_network.state_dict()
        policy_net_state_dict = self.q_network.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*self.tau + target_net_state_dict[key]*(1-self.tau)
        self.target_q_network.load_state_dict(target_net_state_dict)


        return loss.item()


## Part 1: Balancing a pole with a cart

First, we'll test both algorithms on a very simple toy system: the cartpole. Eventhough it's very low dimensional (state=4, action=2), this task is nontrival because it is underactuated. Nevertheless after a few thousand episodes our policy shouldn't have a problem! 

In [ ]:
# Optimization hyperparameters
lr = 0.005
weight_decay = 0

In [ ]:
env_name = "LunarLander-v3" 
#env_name = 'MountainCar-v0'
e = Pytorch_Gym_Env(env_name)
state_dim = e.observation_space.shape[0]
action_dim = e.action_space.n

In [ ]:
# Choose what agent to use
agent = PPO(state_dim, action_dim, lr=lr, weight_decay=weight_decay)

total_episodes = 0
print(agent) # Let's take a look at what we're working with...

In [ ]:
# Create a 
gen = Generator(e, agent)

## Let's do this!!

Below is the loop to train and evaluate your agent. You can play around with the number of iterations to run, and the number of rollouts per iteration. 

You can rerun this cell multiple times to keep training your model for more episodes. In any case, it shouldn't take more than 30 min to an 1 hour to train. (training never took me more than 5 min). HINT: Keep an eye on the eval_reward, it'll be pretty noisy, but if that should be slowly increasing.

In [ ]:
num_iter = 5000
num_train = 10
num_eval = 10 # dont change this
rewards_list = []
episodes_list = []
for itr in range(num_iter):
    #agent.model.epsilon = epsilon * epsilon_decay ** (total_episodes / epsilon_decay_episodes)
    #print('** Iteration {}/{} **'.format(itr+1, num_iter))
    train_reward, train_loss = run_iteration('train', num_train, agent, gen)
    eval_reward, _ = run_iteration('eval', num_eval, agent, gen)
    total_episodes += num_train
    print('Ep:{}:'.format(total_episodes))
    print('reward={:.3f}'.format(train_reward))
    print(f'train loss: {train_loss}')
    print('eval={:.3f}'.format(eval_reward))
    rewards_list.append(eval_reward)
    episodes_list.append(total_episodes)
    #CartPole-v1 LunarLander-v3
    if eval_reward > 199 and env_name == 'LunarLander-v3': # dont change this
        print('Success!!! You have solved cartpole task! Time for a bigger challenge!')
        break
    
    # save model
print('Done')

In [ ]:
# You can visualize your policy at any time
vis_gen = Generator(Pytorch_Gym_Env(env_name, render_mode="human"), agent)
run_iteration('eval', 1, agent, vis_gen, render=True)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(A3C_episodes_list, A3C_rewards_list, label="A3C")
plt.plot(REINFORCE_episodes_list, REINFORCE_rewards_list, label="REINFORCE")
plt.plot(DQN_episodes_list, DQN_rewards_list, label="DQN")
plt.plot(PPO_episodes_list, PPO_rewards_list, label="PPO")
plt.xlabel('Episodes')
plt.ylabel('Evaluation Rewards')
plt.title("Performances on CartPole")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print(max(PPO_rewards_list))
print(max(A3C_rewards_list))
print(max(REINFORCE_rewards_list))
print(max(DQN_rewards_list))